**Общая информация**

**Срок сдачи:** 21 октября 2020, 08:30 

**Штраф за опоздание:** по 1 баллу за 24 часа задержки. Через 5 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0220, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
2. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
#from sklearn.datasets import fetch_mldata, fetch_20newsgroups
from sklearn.datasets import fetch_openml, fetch_20newsgroups

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KDTree
from sklearn.metrics import accuracy_score
from sklearn.metrics import pairwise_distances

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html) 

// дефолтно используется евклидова метрика

In [2]:
class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors, algorithm='brute'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm
    
    def fit(self, X, y):
        self.X_tr = X
        self.Y_tr = y
    
    def predict(self, X):
        num_test = X.shape[0]
        Y_pred = np.zeros(num_test, dtype = self.Y_tr.dtype)
        
        if self.algorithm == 'brute':
            dist = pairwise_distances(X, self.X_tr)
            pred = np.zeros(num_test * self.n_neighbors, dtype = self.Y_tr.dtype).reshape(num_test, self.n_neighbors)
        elif self.algorithm == 'kd_tree':
            tree = KDTree(self.X_tr)
            pred = tree.query(X, k=self.n_neighbors, return_distance=False)
            
        for i in range(len(X)):
            for j in range(self.n_neighbors):
                if self.algorithm == 'brute':
                    pred[i, j] = self.Y_tr[np.argmin(dist[i])]
                    dist[i, np.argmin(dist[i])] = dist[i, np.argmax(dist[i])]
                elif self.algorithm == 'kd_tree':
                    pred[i,j] = self.Y_tr[pred[i,j]]  
            Y_pred[i] = pred[i, np.argmax(pred[i].count(x) for x in pred[i])]
        
        return Y_pred

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [3]:
iris = datasets.load_iris()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [5]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [6]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [7]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
print(accuracy_score(y_test, my_clf_pred))
print(accuracy_score(y_test, sklearn_pred))
assert abs(accuracy_score(y_test, my_clf_pred) - accuracy_score(y_test, sklearn_pred) )<0.005, "Score must be simillar"

0.9333333333333333
0.9333333333333333


**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [8]:
%time clf.fit(X_train, y_train)

CPU times: user 1.19 ms, sys: 332 µs, total: 1.52 ms
Wall time: 1.18 ms


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [9]:
%time my_clf.fit(X_train, y_train)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.3 µs


In [10]:
%time clf.predict(X_test)

CPU times: user 2.33 ms, sys: 649 µs, total: 2.98 ms
Wall time: 2.52 ms


array([2, 1, 0, 1, 1, 0, 1, 1, 0, 2, 2, 0, 2, 0, 1])

In [11]:
%time my_clf.predict(X_test)

CPU times: user 0 ns, sys: 2.68 ms, total: 2.68 ms
Wall time: 2.08 ms


array([2, 1, 0, 1, 1, 0, 1, 1, 0, 2, 2, 0, 2, 0, 1])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [12]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [13]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [14]:
%time clf.fit(X_train, y_train)

CPU times: user 1.02 ms, sys: 285 µs, total: 1.31 ms
Wall time: 1.11 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [15]:
%time my_clf.fit(X_train, y_train)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.87 µs


In [16]:
%time clf.predict(X_test)

CPU times: user 4.56 ms, sys: 119 µs, total: 4.68 ms
Wall time: 3.35 ms


array([0, 2, 2, 1, 0, 0, 1, 1, 2, 0, 1, 2, 0, 1, 1])

In [17]:
%time my_clf.predict(X_test)

CPU times: user 965 µs, sys: 270 µs, total: 1.23 ms
Wall time: 943 µs


array([0, 2, 2, 1, 0, 0, 1, 1, 2, 0, 1, 2, 0, 1, 1])

In [18]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
print(accuracy_score(y_test, my_clf_pred))
print(accuracy_score(y_test, sklearn_pred))
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

0.9333333333333333
0.9333333333333333


**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [19]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [20]:
data = newsgroups['data']
target = newsgroups['target']

Преобразуйте текстовые данные из data с помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Словарь можно ограничить по частотности.

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
vect = CountVectorizer()
vect_data = vect.fit_transform(data)

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [23]:
#from sklearn.cross_validation import KFold
from sklearn.model_selection import KFold

In [24]:
cross_val = KFold(n_splits=3)
for train_index, test_index in cross_val.split(vect_data):
    X_train, X_test = vect_data[train_index], vect_data[test_index]
    y_train, y_test = target[train_index], target[test_index]

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальную точность в среднем на валидации на 3 фолдах).
Постройте график зависимости средней точности от количества соседей. Можно рассмотреть число соседей от 1 до 10.

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf векторизацию( sklearn.feature_extraction.text.TfidfVectorizer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [25]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим точность вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.